In [6]:
# Classical Shadows (X/Y/Z random local measurements) para um par EPR (Bell)
# Requer: qiskit
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

# --- utilitários ---
def single_qubit_projector(bit, basis):
    zero = np.array([1,0], dtype=complex)
    one  = np.array([0,1], dtype=complex)
    if basis == 'Z':
        vec = zero if bit == '0' else one
    elif basis == 'X':
        plus = (zero + one)/np.sqrt(2)
        minus = (zero - one)/np.sqrt(2)
        vec = plus if bit == '0' else minus
    elif basis == 'Y':
        plus_i = (zero + 1j*one)/np.sqrt(2)
        minus_i = (zero - 1j*one)/np.sqrt(2)
        vec = plus_i if bit == '0' else minus_i
    else:
        raise ValueError("basis must be 'X','Y' or 'Z'")
    return np.outer(vec, np.conj(vec))

def single_qubit_inverse_map_from_proj(proj):
    return 3 * proj - np.eye(2, dtype=complex)

def tensor_kron(mats):
    out = mats[0]
    for m in mats[1:]:
        out = np.kron(out, m)
    return out

# --- parâmetros ---
n_qubits = 2
n_shadows = 2000      # ajuste conforme máquina/tempo
shots_per_shadow = 1  # geralmente 1 por shadow no protocolo clássico
seed = 1717

sim = AerSimulator()
rng = np.random.default_rng(seed)
bases = ['X','Y','Z']

# prepara EPR (Bell) entre qubit 0 e 1
def prepare_epr():
    qc = QuantumCircuit(n_qubits)
    qc.h(0)
    qc.cx(0,1)
    return qc

def basis_rotation(qc, qubit, basis):
    # rotações que mapeiam medição em basis -> medição em Z
    if basis == 'X':
        qc.h(qubit)
    elif basis == 'Y':
        qc.sdg(qubit)
        qc.h(qubit)
    elif basis == 'Z':
        pass

estimators = []
for _ in range(n_shadows):
    chosen_bases = [rng.choice(bases) for _ in range(n_qubits)]
    qc = prepare_epr()
    for q, b in enumerate(chosen_bases):
        basis_rotation(qc, q, b)
    qc.save_statevector()
    qc = transpile(qc, sim)
    result = sim.run(qc, shots=shots_per_shadow).result()
    sv = result.data()['statevector']
    counts = Statevector(sv).sample_counts(shots_per_shadow, seed=rng.integers(0,2**30))
    for bitstr, cnt in counts.items():
        bitstr_rev = bitstr[::-1]  # ajustar endian
        for _ in range(cnt):
            local_mats = []
            for q in range(n_qubits):
                b = chosen_bases[q]
                outcome = bitstr_rev[q]
                proj = single_qubit_projector(outcome, b)
                inv = single_qubit_inverse_map_from_proj(proj)
                local_mats.append(inv)
            estimator = tensor_kron(local_mats)
            estimators.append(estimator)

rho_hat = sum(estimators) / len(estimators)

# estado real (para comparação)
rho_true = Statevector(prepare_epr()).to_density_matrix()
rho_true = np.array(rho_true.data)

print("Trace rho_hat:", np.trace(rho_hat))
print("Purity rho_hat:", np.real_if_close(np.trace(rho_hat @ rho_hat)))
print("Fidelity com estado ideal:", fidelity(rho_true, rho_hat))
print("rho_hat (matriz):")
np.set_printoptions(precision=4, suppress=True)
print(rho_hat)


NameError: name 'transpile' is not defined

In [3]:
import numpy as np
import qiskit

def Minv(N, X):
    '''Inverso do canal médio para shadow tomography'''
    dim = 2**N
    return (dim + 1) * X - np.eye(dim)

# Número de qubits do sistema (EPR é 2 qubits)
N = 2

# Número de shadows e shots
nShadows = 1000
reps = 50

rng = np.random.default_rng(1717)

# Gera Cliffords aleatórios para o sistema de N qubits
cliffords = [qiskit.quantum_info.random_clifford(N, seed=rng) for _ in range(nShadows)]

# Circuito que prepara o estado EPR (Bell Phi+)
def prepare_epr():
    qc = qiskit.QuantumCircuit(N)
    qc.h(0)
    qc.cx(0, 1)
    return qc

# Estado ideal para comparação
qc_epr = prepare_epr()
state_epr = qiskit.quantum_info.Statevector(qc_epr)

# Lista para armazenar resultados das medidas
results = []

for cliff in cliffords:
    # Prepara circuito para o estado EPR
    qc = prepare_epr()
    
    # Aplica o Clifford (rotação aleatória da base)
    qc = qc.compose(cliff.to_circuit())
    
    # Simula amostras de medidas no computational basis
    counts = qiskit.quantum_info.Statevector(qc).sample_counts(reps)
    results.append(counts)

# Reconstrução da matriz densidade a partir dos resultados
rho_shadow = np.zeros((2**N, 2**N), dtype=complex)

for cliff, res in zip(cliffords, results):
    mat = cliff.adjoint().to_matrix()
    for bit, count in res.items():
        vec = mat[:, int(bit, 2)]
        proj = np.outer(vec, np.conj(vec))
        rho_shadow += Minv(N, proj) * count

# Normalização
rho_shadow /= (nShadows * reps)
rho_shadow *= 2**N

print("Matriz densidade estimada (rho_shadow):")
print(rho_shadow)

# Para comparação, matriz densidade exata do estado EPR puro
rho_exact = state_epr.to_density_matrix()
print("\nMatriz densidade exata do estado EPR:")
print(rho_exact)

# Medir distância de traço entre estimado e exato (opcional)
def trace_dist(rho1, rho2):
    diff = rho1 - rho2
    sqrt_diff = scipy.linalg.sqrtm(diff.conj().T @ diff)
    return 0.5 * np.trace(sqrt_diff).real

import scipy.linalg
dist = trace_dist(rho_shadow, rho_exact)
print(f"\nDistância de traço entre estimado e exato: {dist:.6f}")


Matriz densidade estimada (rho_shadow):
[[ 1.9462+1.72933522e-18j -0.0148-1.52000000e-02j -0.0268-1.38000000e-02j
   2.12  -1.64000000e-02j]
 [-0.0148+1.52000000e-02j  0.0402+4.60455165e-19j  0.03  -1.08000000e-02j
  -0.0272-1.46000000e-02j]
 [-0.0268+1.38000000e-02j  0.03  +1.08000000e-02j  0.0898+1.01277452e-18j
  -0.0124+1.64000000e-02j]
 [ 2.12  +1.64000000e-02j -0.0272+1.46000000e-02j -0.0124-1.64000000e-02j
   1.9238+8.09501682e-19j]]


AttributeError: 'Statevector' object has no attribute 'to_density_matrix'